### 使用CV扫描脸部，读取数据点


In [1]:
from scipy.spatial import distance as dist
import numpy as np
import dlib
import cv2

# 定义视频源的URL
# url = "http://192.168.137.240:8081/"#树莓派1，用于面部抓取疲劳值
# url = "http://192.168.137.32:8081/"#树莓派2，用于面部识别
url = "http://mirror.aarnet.edu.au/pub/TED-talks/911Mothers_2010W-480p.mp4"#测试视频用

# 创建视频捕获对象
cap = cv2.VideoCapture(url)

# 检查视频捕获对象是否成功打开
if not cap.isOpened():
    print("无法打开视频源")
    exit()

#定义脸上采集点
FACIAL_LANDMARKS_68_IDXS = dict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))
])

#眨眼计算函数(待修改！！！)
def eye_aspect_ratio(eye):
    # 计算距离，竖直的
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # 计算距离，水平的
    C = dist.euclidean(eye[0], eye[3])
    # ear值
    ear = (A + B) / (2.0 * C)
    return ear

#打哈欠计算函数
def mouth_aspect_ratio(mouth):
    # 计算距离，竖直的
    A = dist.euclidean(mouth[1], mouth[5])
    B = dist.euclidean(mouth[2], mouth[4])
    # 计算距离，水平的
    C = dist.euclidean(mouth[0], mouth[3])
    # ear值
    ear = (A + B) / (2.0 * C)
    return ear

#关键点定位
def shape_to_np(shape, dtype="int"):
    # 创建68*2
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)
    # 遍历每一个关键点
    # 得到坐标
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

# 设置判断参数
EYE_AR_THRESH = 0.3  # ear小于0.3判断为闭眼
EYE_AR_CONSEC_FRAMES = 3  # 连续三帧ear都小于0.3判断为眨眼

# 初始化计数器
COUNTER = 0
TOTAL = 0

#加载dlib库中的人脸检测与关键点定位
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('.\shape_predictor_68_face_landmarks.dat')

#分别取两个眼睛区域
(lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]


while True:
    # 读取视频帧
    ret, frame = cap.read()

    # 检查是否成功读取帧
    if not ret:
        print("无法获取视频帧")
        break
        
    (h, w) = frame.shape[:2]
    width=1200
    r = width / float(w)
    dim = (width, int(h * r))
    frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 检测人脸
    rects = detector(gray, 0)

    # 遍历每一个检测到的人脸
    for rect in rects:
        # 获取坐标
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)

        # 分别计算ear值
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # 算一个平均的
        ear = (leftEAR + rightEAR) / 2.0

        # 绘制眼睛区域
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        # 检查是否满足阈值
        if ear < EYE_AR_THRESH:
            COUNTER += 1

        else:
            # 如果连续几帧都是闭眼的，总数算一次
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1

            # 重置
            COUNTER = 0

        # 显示
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(10) & 0xFF
    
    # 按下 'q' 键退出循环
    if cv2.waitKey(1) == ord('q'):
        break

# 释放视频捕获对象和窗口
cap.release()
cv2.destroyAllWindows()
